The goal here is to construct a fake stellar stream using a density distribution. Here we recreate a stellar stream using its density distribution.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.typing as npt

from stellar_stream import StellarStream

In [ ]:
def stream_from_density(centers, dens, size, phi2_sigma, vlos_sigma, rng=None):
    if rng==None:
        rng = np.random.default_rng()
    
    phi1_pieces = []
    phi2_pieces = []
    vlos_pieces = []
    
    mu = 0
    dx = np.diff(centers)[0]
    #piece_size = size / len(centers)
    
    full_size = 0
    
    for center, density in zip(centers, dens):
        stars_in_piece = int(size * density)
        full_size += stars_in_piece

        phi1 = rng.uniform(center, center + dx, stars_in_piece)
        phi2 = rng.normal(mu, phi2_sigma, stars_in_piece)
        vlos = rng.normal(mu, vlos_sigma, stars_in_piece)
        
        phi1_pieces.append(phi1)
        phi2_pieces.append(phi2)
        vlos_pieces.append(vlos)
    
    phi1_array = np.concatenate(phi1_pieces)
    phi2_array = np.concatenate(phi2_pieces)
    vlos_array = np.concatenate(vlos_pieces)
    print(full_size)
    return phi1_array, phi2_array, vlos_array


In [ ]:
from pathlib import Path
p = Path('../../data/sample')

simulations = {}

for sim_data in p.glob('*.npz'):
    simulation_number = sim_data.name.removeprefix('simdatag').removesuffix('.npz')
    simulations[simulation_number] = (StellarStream.from_simulation(simulation_number, sim_data)
                                            # .select("restrict_phi1", phi1_lim=(-110, 10))
                                            # .select("restrict_phi2", phi2_lim=15.0)
                                            # .run_analysis(name="detrend", polynomial_fit_degree=4)
                                            .run_analysis(name="gaussian_process_detrend", n_bins=40)
                                            # .select("restrict_phi1", phi1_lim=(-100, 0))
                                            # .select("restrict_phi2", phi2_lim=6.0)
    )

In [ ]:
centers, dens = simulations['714'].density_phi1(precision=1)
size = len(simulations['714'].phi1)
phi2_sigma,vlos_sigma = 1, 5

phi1, phi2, vlos = stream_from_density(centers, dens, size, phi2_sigma, vlos_sigma)

S = (StellarStream.from_catalog(phi1, phi2, vlos, '714 recon'))

In [ ]:
S.plot_stream()

In [ ]:
simulations['714'].plot_stream()

In [ ]:
S.plot_density()

In [ ]:
simulations['714'].plot_density()